In [1]:
using PyCall
@pyimport nltk

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2499: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [10]:
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

tokenize (generic function with 1 method)

In [2]:
include("load_embeddings.jl")

load_embeddings (generic function with 1 method)

In [3]:
LL,word_index,indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_index|> typeof |> println
indexed_words|> typeof |> println

(50,268810)

In [8]:
typealias Embedding Vector{Float64}
typealias Embeddings Matrix{Float64}

type RAE
    L::Matrix{Float64}
    word_index::Dict{String,BitVector}
    indexed_words::Vector{String}
    
    W_e::Matrix{Float64}
    W_d::Matrix{Float64}
   
end


function RAE(L::Matrix{Float64}, word_index::Dict{String,BitVector}, indexed_words::Vector{String})
    emb_width = size(L,1)
    
    RAE(L,word_index, indexed_words,
        0.01*randn(emb_width,emb_width*2+1),
        0.01*randn(emb_width*2,emb_width) )
end

function eval_embedding(rae::RAE, input::String)
    k=word_index[input]
    rae.L*eval_embedding
end

function eval_embeddings(rae::RAE, c_is::Embeddings, c_js::Embeddings)
    @assert size(c_is)==size(c_js)
    bias_in = ones(1,size([c_is], 2))
    tanh(rae.W_e*[c_is;c_js;bias_in])
end

function eval_scores(rae::RAE, input_A::Embeddings, input_B::Embeddings)
    pp=eval_embedding(rae, input_A, input_B)
    
    bias_in = ones(1,size(pp, 2))
    ĉ_ijs = tanh(rae.W_d*[pp;bias_in])
    
end


eval_score (generic function with 1 method)

In [7]:
function word_adjancy_matrix(sentence_len::Int)
    #TODO: Replace this whole adjancy matrix with a linked list
    [(ii==jj+1) || (ii==jj-1)  for ii in 1:sentence_len, jj in 1:sentence_len]
end



function eval_to_tree(rr::RNN, sentence::AbstractArray{String})
    backlookup = sentence
    ss = hcat([get_embedding(embeddings, token) for token in backlookup]...)
    AA = word_adjancy_matrix(size(ss,2))
    
    score_total = 0.0
    while(any(AA))
        iis, jjs = findn(AA)

        pps = eval_embedding(rr, ss[:,iis],ss[:,jjs])
        scores = eval_score(rr, pps)
        best_pair_ind = indmax(scores)
        
         
        ii_best, jj_best = sort([iis[best_pair_ind], jjs[best_pair_ind]])
        #The above makes a more readable output, but doesn't do anything useful,
        #it is same as below:
        #ii_best = iis[best_pair_ind]
        #jj_best = jjs[best_pair_ind]
        pp_best = pps[:,best_pair_ind]
        score_total+=scores[best_pair_ind]
        
        ss = [ss pp_best]
        backlookup = [backlookup, (backlookup[ii_best],backlookup[jj_best])]
        
        #Adjust Adjacency Matrix

        AA[ii_best,jj_best] = false
        AA[jj_best,ii_best] = false
        AA = [AA; AA[ii_best,:] | AA[jj_best,:]] # Add row
        AA = [AA AA[:,ii_best] | AA[:,jj_best]] # Add col
        AA[ii_best,:] = AA[jj_best,:] = false
        AA[:,ii_best] = AA[:,jj_best] = false #Remove anything that was adjacent to old

    end
    tree = backlookup[end]
    embedding = ss[:,end]
    tree, embedding, score_total
end
    

eval_to_tree (generic function with 1 method)

tokenize (generic function with 1 method)

In [11]:
sentence = "The house has a window."
sentence_toks = tokenize(sentence)
rr = RNN(LL,3)
eval_to_tree(rr,sentence_toks)




LoadError: input_width not defined
while loading In[11], in expression starting on line 3

In [ ]:
function bpts(rr::RNN, p)
    δp = 

In [12]:
function get_all_pos_tags()
    lines = split(open(readall,"brown_tags.txt"),"\n")
    desc_lines = filter(line -> contains(line,"||"), lines)
    tags = [split(line,"||")[1] for line in desc_lines]
end
pos_tags = get_all_pos_tags();

In [13]:
length(pos_tags)

226

In [16]:
readall(`ls`)

"ACL2012_wordvectors.txt\nbrown_tags.txt\nembeddings-scaled.EMBEDDING_SIZE=200.txt\nembeddings-scaled.EMBEDDING_SIZE=50.txt\nFactoristation.ipynb\ngit.ipynb\nhypernym_distance.ipynb\nLets build a NN for practice.ipynb\nload_embeddings.jl\nMacro experiments.ipynb\nParseToEmbeedings.ipynb\nSentence splitting and POS.ipynb\nTokenize_to_embeddings.ipynb\nwordVectors.txt\n"